Importing Libaries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Input, Dropout, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Reshape, concatenate 


Setting up on-demand loading

In [ ]:
root = '/content/drive/MyDrive/Final Year Project  /Data'#training data comes from multiple sources and they are stored in the 'data' folder

In [ ]:
input_dirs = [root+'/G1020/Images', root+'/ORIGA/Images']
mask_dirs = [root+'/G1020/ROI', root+'/ORIGA/ROI']

In [ ]:
#list of all image file paths
input_files = []
for dir in input_dirs:
    input_files.extend([os.path.join(dir, f) for f in sorted(os.listdir(dir))])

#list of all mask file paths
mask_files = []
for dir in mask_dirs:
    mask_files.extend([os.path.join(dir, f) for f in sorted(os.listdir(dir))])

#DataFrame with both file paths
data = pd.DataFrame({'input': input_files, 'mask': mask_files})
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
tf.random.set_seed(42)
# Define the data generator for the input images
image_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

#data generator for the mask images is seperate
#as rescale would damage the mask
mask_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)


# Create the generator for the input images
image_generator = image_datagen.flow_from_dataframe(
    dataframe=data,
    x_col='input',
    y_col=None,
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
)

# Create the generator for the mask images
mask_generator = mask_datagen.flow_from_dataframe(
    dataframe=data,
    x_col='mask',
    y_col=None,
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
    color_mode='grayscale'
)
train_generator = zip(image_generator, mask_generator)      

In [ ]:
train_generator = custom_image_mask_generator(train_data, batch_size=32, target_size=(256, 256))
test_generator = custom_image_mask_generator(test_data, batch_size=32, target_size=(256, 256))

Designing the UNet architeture

In [ ]:
inputs = Input(shape=(256, 256, 3))

# Contracting path
conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

# Expansive path
up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv7))
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)

up9 = UpSampling2D(size=(2,2))(conv8)
up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up9)
merge9 = concatenate([conv1,up9], axis=3)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)

conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

# Define the model
model = Model(inputs=inputs, outputs=conv10)


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
          steps_per_epoch=len(data) // 32,
          epochs=10)

In [ ]:
keras.models.save_model(model, '/content/drive/MyDrive/Final Year Project  /Models/ROIExstractor.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Final Year Project  /Models/ROIExstractor.h5')

In [ ]:
model.summary()

In [ ]:
image = cv2.imread(mask_files[0])

height, width = image.shape[:2]

aspect_ratio = float(width) / float(height)

if height > width:
    new_height = 256
    new_width = int(new_height * aspect_ratio)
else:
    new_width = 256
    new_height = int(new_width / aspect_ratio)

resized_img = cv2.resize(image, (new_width, new_height))

background = np.zeros((256, 256, 3), dtype=np.uint8)

x_offset = int((256 - new_width) / 2)
y_offset = int((256 - new_height) / 2)

background[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_img
image = background

image = np.expand_dims(image, axis=0)

In [ ]:
pred = model.predict(image)